En esta práctica utilizaremos el dataset MNIST, compuesto por 40000 imágenes (de 28x28 pixeles) correspondientes a distintas versiones digitalizadas de los dígitos 0, ..., 9. El objetivo es clasificar correctamente cada una de las imágenes, utilizando el valor de los 784 pixeles. La información está disponible en el fichero 'train.csv'; la primera columna es la etiqueta del dígito (de 0 a 9) y el resto de columnas son los valores de los pixeles.

In [ ]:
train <- read.csv('train.csv')

In [ ]:
# Clase (dígito del 0 al 9 a predecir)
y <- train[,1]; 
# 784 columnas con los datos de los pixeles
x <- train[,-1]

In [ ]:
# Cada dígito está igualmente representado en la muestra
hist(y,breaks = -0.5:9.5)

In [ ]:
# Dibujamos los 6 primeros dígitos de la base de datos
par(mfrow = c(2,3))
for (i in 1:6) {
    image(matrix(as.matrix(x[i,1:784]), nrow = 28, ncol = 28))
    title(y[i])
}

Este problema se encuandra dentro de la clasificación multi-categoría (tenemos 10 posibles clases a predicir para cada dato). Para simplificar estos problemas, se suelen consideran tantos problemas binarios (predecir sí o no) independientes como categorías se tengan. Es decir, un modelo para clasificar 0 (si o no), otro para 1, etc. En este ejemplo, consideramos el dígito '9' y el objetivo es predecir si una imagen es 9 o no.

In [ ]:
# Seleccionamos un dígito concreto para tratar de predecirlo. Por ejemplo el nueve
# y9 = 1 si el dígito es 9 y 0 en caso contrario
y9 <- y;
y9[which(y != 9)] <- 0 ; y9[which(y == 9)] <- 1

In [ ]:
# Construimos un dataframe para entrenar el modelo
dat <- data.frame(y9,x)

Como el tamaño de la muestra es muy grande (40000 imagenes), seleccionamos una muestra para entrenar; por ejemplo, los 1000 primeros dígitos y el resto lo usaremos para validar.

In [ ]:
i<- 1:1000;
#Random sampling
#i<-sample(1:dim(x)[1],1000);
datT <- dat[i,];
datt<- dat[-i,];

Aunque no es el modelo adecuado, construimos primero un modelo de regresión lineal para estos datos.

In [ ]:
# Entrenamos primero un modelo lineal
model <- lm(y9~., data = datT)
out <- model$fitted.values
# El histograma muestra valores predichos (incluyendo negativos y mayores que 1)
hist(out)

In [ ]:
# Hacemos binaria la salida considerando el umbral 0.5 (a modo de probabilidad)
outbin <- as.double(out > 0.5)
# Tasa de acierto (en entrenamiento)
100*sum(diag(table(datT[,1], outbin))) / length(outbin)

In [ ]:
# Tasa de acierto de test
out1<- predict(object=model, newdata = datt);
outbin1 <- as.double(out1 > 0.5)
100*sum(diag(table(datt[,1], outbin1))) / length(outbin1)

Ahora construimos un modelo de regresión logística utilizando la función 'glm' con la familia 'family = binomial(link = "logit")'

In [ ]:
model1 <- glm(y9~., data = datT, family = binomial(link = "logit"))
# Problemas de convergencia con tantas variables
# Coefficients: (138 not defined because of singularities)
out2 <- model1$fitted.values
outbin2 <- as.double(out2 > 0.5)
100*sum(diag(table(datT[,1], outbin2))) / length(outbin2)

In [ ]:
hist(out2)

In [ ]:
# Tasa de acierto (test)
out3<- predict(object=model1, newdata = datt);
outbin3 <- as.double(out3 > 0.5)
100*sum(diag(table(datt[,1], outbin3))) / length(outbin3)

Las inestabilidades numéricas se producen por la alta dimensionalidad de los datos. Algunos de los coeficientes resultan redundantes y el proceso de optimización no converge. # (195 not defined because of singularities).

In [ ]:
summary(model1)

Una solución a este problema es reducir la dimensionalidad del conjunto de predictores. Para ello existen técnicas eficientes (como los métodos de regularización o las Componentes Principales) que se verán más adelante. En esta práctica utilizamos un entresacado de información, considerando sólo uno de cada 20 pixeles.

In [ ]:
#Modelo de regresión lineal
j<-seq(1,784,20)
model <- lm(y9~., data = datT[,j])
out <- model$fitted.values
outbin <- as.double(out > 0.5)
100*sum(diag(table(datT[,1], outbin))) / length(outbin)
out1<- predict(object=model, newdata = datt[,j]);
outbin1 <- as.double(out1 > 0.5)
100*sum(diag(table(datt[,1], outbin1))) / length(outbin1)

In [ ]:
#Regresión logística
j<-seq(1,784,20)
model <- glm(y9~., data = datT[,j], family = binomial(link = "logit"))
out <- model$fitted.values
outbin <- as.double(out > 0.5)
100*sum(diag(table(datT[,1], outbin))) / length(outbin)
out1<- predict(object=model, newdata = datt[,j]);
outbin1 <- as.double(out1 > 0.5)
100*sum(diag(table(datt[,1], outbin1))) / length(outbin1)

In [ ]:
hist(out)

In [ ]:
i<- 1:10000;
#Random sampling
#i<-sample(1:dim(x)[1],1000);
datT <- dat[i,];
datt<- dat[-i,];

j<-seq(1,784,20)
model <- glm(y9~., data = datT[,j], family = binomial(link = "logit"))
out <- model$fitted.values
outbin <- as.double(out > 0.5)
100*sum(diag(table(datT[,1], outbin))) / length(outbin)
out1<- predict(object=model, newdata = datt[,j]);
outbin1 <- as.double(out1 > 0.5)
100*sum(diag(table(datt[,1], outbin1))) / length(outbin1)

PRACTICA 1: Construir un modelo de clasificación para cada dígito, y obtener una ranking de los dígitos en base a su capacidad predictiva (el primero el que mejor se predice, etc.). Para evaluar la capacidad predictiva considerando el error de test obtenido al separar aleatoriamente la muestra en un conjunto de train (n=10000) y el resto de test. Nota: usar la función 'sample'. Nota2: Elige un entresacado óptimo para que la capacidad predictiva no se reduzca y se reduzca la dimensión el máximo posible.

PRACTICA 2: Tener en cuenta la variabilidad del error de test a la hora de construir el ranking anterior. Para ello, además de calcular la tasa de acierto para cada dígito, considerar también un "intervalo de confianza" obtenido como la dispersión (desviación típica) de 10 medidas de test obtenidas con 10 muestras aleatorias distintas. ¿Existe algún dígito que pueda predecirse significativamente mejor que los demás?

TRABAJO EXTRA: Hasta ahora hemos utilizado la tasa de acierto como medida de validación de los clasificadores. Para ello, las predicciones probabilísticas se tiene que convertir a binarias (utilizando un umbral para al probabilidad). Sin embargo, existen otras medidas más generales que consideran el carácter probabilístico de la predicción. Leer https://es.wikipedia.org/wiki/Curva_ROC Practicar con el paquete 'AUC' #install.packages("AUC") #library("AUC") #roc<-roc(out,as.factor(datT[,1])) #auc(roc) #plot(roc)